# Calculation of the fn-coefficients

In the following, a simple example on how to manually evaluate the fn-coefficients is given. 

The ground is hereby defined as a Lambertian-surface and the covering layer is assumed to consist of Rayleigh-particles.  

Within the following calculation, the symbols and functions are defined as presented in the Theory-chapter of the documentation.

## Definition of the surface and volume scattering distribution functions:
- $R_0$ denotes the diffuse albedo of the surface)
- $\cos(\Theta)$ denotes the cosine of the scattering-angle which is defined as:
  $\qquad \cos(\Theta) = \cos(\theta)\cos(\theta_{ex}) + \sin(\theta)\sin(\theta_{ex})\cos(\phi - \phi_{ex})$

$BRDF(\theta, \phi, \theta_{ex},\phi_{ex}) = \frac{R_0}{\pi}$

$p(\theta, \phi, \theta_{ex},\phi_{ex}) = \frac{3}{16\pi} (1+\cos(\Theta)^2)$



## Manual evaluation of the fn-coefficients:

$INT := \int_0^{2\pi} p(\theta_0, \phi_0, \theta,\phi) * BRDF(\pi-\theta, \phi, \theta_{ex},\phi_{ex}) d\phi$

$\phantom{INT} = \frac{3 R_0}{16 \pi^2} \int_{0}^{2\pi}  (1+[\cos(\theta_0)\cos(\theta) + \sin(\theta_0)\sin(\theta)\cos(\phi_0 - \phi)]^2) d\phi$

$\phantom{INT} = \frac{3 R_0}{16 \pi^2} \int_0^{2\pi} (1+ \mu_0^2 \mu^2 + 2 \mu_0 \mu \sin(\theta_0) \sin(\theta) \cos(\phi_0 - \phi) + (1-\mu_0)^2(1-\mu)^2 \cos(\phi_0 - \phi)^2 d\phi$


where the shorthand-notation $\mu_x = \cos(\theta_x)$ has been introduced.
The above integral can now easily be solved by noticing:

$\int_0^{2\pi} \cos(\phi_0 - \phi)^n d\phi = \left\lbrace \begin{matrix} 2 \pi & \textrm{for } n=0 \\ 0 & \textrm{for } n=1 \\ \pi  & \textrm{for } n=2 \end{matrix} \right.$
    
Inserting the above solution and using some algebraic manipulations we therefore find:

$INT = \frac{3 R_0}{16\pi} \Big[ (3-\mu_0^2) + (3 \mu_0 -1) \mu^2 \Big] := R_0 ~\sum_{n=0}^2 f_n \, \mu^n$

And so the ``fn`` coefficients are given by:

$$\quad f_0 = \frac{3(3-\mu_0^2)}{16\pi} \qquad f_1 = 0 \qquad f_2 = \frac{3(3 \mu_0 -1)}{16\pi} \qquad f_n = 0 \, \forall \, n>2$$

## Evaluation of the fn-coefficients using the RT1-module

In [1]:
from rt1_model import RT1, volume, surface
import sympy as sp
# enable printing sympy equations via latex-equation-rendering
sp.init_printing(use_latex='mathjax')

## Definition of volume and surface scattering distributions

In [11]:
SRF = surface.Isotropic()
V = volume.Rayleigh()

R = RT1(V=V, SRF=SRF)
R.set_monostatic(p_0=0)

## Check the evaluated fn-coefficients

In [12]:
sp.sympify(R._fn)

⎡          2                             2                2    ⎤
⎢0.1875⋅sin (θ₀)   0.375       0.1875⋅sin (θ₀)   0.375⋅cos (θ₀)⎥
⎢─────────────── + ─────, 0, - ─────────────── + ──────────────⎥
⎣       π            π                π                π       ⎦

## Apply some simplifications to see that the coefficients are really equal

In [13]:
[sp.trigsimp(fi).xreplace({sp.sin('theta_0')**2 : 1.- sp.Symbol("mu_0")**2}).nsimplify().simplify() for fi in R._fn]

⎡  ⎛      2⎞       ⎛    2    ⎞⎤
⎢3⋅⎝3 - μ₀ ⎠     3⋅⎝3⋅μ₀  - 1⎠⎥
⎢───────────, 0, ─────────────⎥
⎣    16⋅π             16⋅π    ⎦